In [1]:
# Wifi data amalgamation

In [2]:
import pandas as pd
import glob
import os

In [15]:
# Data flow consists of emails. Each email has 1 zipped folder, containing multiple csv files. Each csv files has one sheet.
# Filenames have same title beginning, with different numbers on end. 
data = pd.read_csv("Occupancy_Monitor_Client_Count_-_Most_Buildings_20171024_000515_566.csv", skiprows = 7, encoding = 'macroman', parse_dates = ['Event Time'])

In [16]:
data.head(5)

,Total Count,Event Time,Associated Client Count,Authenticated Client Count
0,total,Mon Oct 23 00:08:35 UTC 2017,8603,8487
1,total,Mon Oct 23 00:13:55 UTC 2017,8515,8402
2,total,Mon Oct 23 00:19:02 UTC 2017,8475,8360
3,total,Mon Oct 23 00:24:10 UTC 2017,8424,8307
4,total,Mon Oct 23 00:29:18 UTC 2017,8352,8239


In [29]:
#have to do it one-by-one if the column does not import correctly as datetime
datetimes=[]
for i in data.index:
    try:
        datetimes.append(pd.to_datetime(data.loc[i]['Event Time']))
    except:
        print(i)
#data['datetimes']=datetimes

279


In [30]:
#something fishy going on here
data.loc[275:282]

,Total Count,Event Time,Associated Client Count,Authenticated Client Count
275,total,Mon Oct 23 23:50:45 UTC 2017,8717,8590
276,total,Mon Oct 23 23:55:56 UTC 2017,8706,8563
277,total,Tue Oct 24 00:00:52 UTC 2017,8637,8500
278,Client Count,nan,NaN,NaN
279,Key,Event Time,Associated Client Count,Authenticated Client Count
280,Lancaster University > Pendle > A Floor,Mon Oct 23 00:08:35 UTC 2017,73,71
281,Lancaster University > Pendle > A Floor,Mon Oct 23 00:13:55 UTC 2017,72,72
282,Lancaster University > Pendle > A Floor,Mon Oct 23 00:19:02 UTC 2017,75,72


In [34]:
#we just ignore, but need to maintain column length
#second, we also create a mask of good rows
datetimes=[]
datetimes_mask=[]
for i in data.index:
    try:
        datetimes.append(pd.to_datetime(data.loc[i]['Event Time']))
        datetimes_mask.append(i)
    except:
        datetimes.append(i)
data['datetimes']=datetimes
datam=data.loc[datetimes_mask]

In [36]:
#still inherist object type, so reassing
datam.dtypes

Total Count                   object
Event Time                    object
Associated Client Count       object
Authenticated Client Count    object
datetimes                     object
dtype: object

In [39]:
datam['datetimes']=pd.DatetimeIndex(datam['datetimes'])

In [40]:
#now AOK
datam.dtypes

Total Count                           object
Event Time                            object
Associated Client Count               object
Authenticated Client Count            object
datetimes                     datetime64[ns]
dtype: object

In [6]:
#no need for this now
#data['Event Time'] = data['Event Time'].str.replace(':',' ')
#data2 = pd.DataFrame([ x.split() for x in data['Event Time'].tolist()])
#data2.head(5)

,0,1,2,3,4,5,6,7
0,Mon,Oct,23,00,08,35,UTC,2017
1,Mon,Oct,23,00,13,55,UTC,2017
2,Mon,Oct,23,00,19,02,UTC,2017
3,Mon,Oct,23,00,24,10,UTC,2017
4,Mon,Oct,23,00,29,18,UTC,2017


In [41]:
datam.columns

Index(['Total Count', 'Event Time', 'Associated Client Count',
       'Authenticated Client Count', 'datetimes'],
      dtype='object')

In [47]:
# Preparation for multi hierarchy
data2=datam[['Total Count', 'Associated Client Count',
       'Authenticated Client Count', 'datetimes']]
dates=pd.DatetimeIndex(datam['datetimes'])
data2['Year']=dates.year
data2['Month']=dates.month
data2['Day']=dates.day
data2['Hour']=dates.hour
data2['Minute']=dates.minute
data2['Second']=dates.second

In [49]:
data2 = data2.set_index(['Year','Month','Day','Hour','Minute','Second'])

In [50]:
data2

Total Count  \
Year   Month Day  Hour Minute Second                                            
2017.0 10.0  23.0 0.0  8.0    35.0                                      total   
                       13.0   55.0                                      total   
                       19.0   2.0                                       total   
                       24.0   10.0                                      total   
                       29.0   18.0                                      total   
                       34.0   37.0                                      total   
                       39.0   30.0                                      total   
                       44.0   42.0                                      total   
                       49.0   49.0                                      total   
                       54.0   53.0                                      total   
                  1.0  0.0    12.0                                      total   
                       5.0    13.0                                      total   
                       10.0   22.0                                      total   
                       15.0   39.0                                      total   
                       20.0   37.0                                      total   
                       25.0   46.0                                      total   
                       31.0   7.0                                       total   
                       35.0   52.0                                      total   
                       41.0   1.0                                       total   
                       46.0   12.0                                      total   
                       51.0   6.0                                       total   
                       56.0   12.0                                      total   
                  2.0  1.0    18.0                                      total   
                       6.0    16.0                                      total   
                       11.0   22.0                                      total   
                       16.0   23.0                                      total   
                       21.0   28.0                                      total   
                       26.0   25.0                                      total   
                       31.0   30.0                                      total   
                       36.0   31.0                                      total   
...                                                                       ...   
                  20.0 29.0   8.0     Lancaster University > SW hse 24-26 > A   
                       34.0   1.0     Lancaster University > SW hse 24-26 > A   
                       39.0   50.0    Lancaster University > SW hse 24-26 > A   
                       44.0   41.0    Lancaster University > SW hse 24-26 > A   
                       49.0   53.0    Lancaster University > SW hse 24-26 > A   
                       55.0   3.0     Lancaster University > SW hse 24-26 > A   
                  21.0 0.0    12.0    Lancaster University > SW hse 24-26 > A   
                       5.0    11.0    Lancaster University > SW hse 24-26 > A   
                       10.0   50.0    Lancaster University > SW hse 24-26 > A   
                       15.0   46.0    Lancaster University > SW hse 24-26 > A   
                       21.0   4.0     Lancaster University > SW hse 24-26 > A   
                       26.0   14.0    Lancaster University > SW hse 24-26 > A   
                       31.0   28.0    Lancaster University > SW hse 24-26 > A   
                       36.0   40.0    Lancaster University > SW hse 24-26 > A   
                       41.0   39.0    Lancaster University > SW hse 24-26 > A   
                       46.0   42.0    Lancaster University > SW hse 24-26 > A   
                       51.0   59.0    Lancaster University > SW hse 24-26 > A   
                